In [ ]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)

### Load image via pytorch

In [ ]:
from torchvision import datasets, transforms

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.CIFAR10(root='./data', train=True,
                              download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=64,
                                           shuffle=True)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Remove all in valid files that is not end with jpg



transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root="data/sample-folder/train", transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


In [ ]:
import pandas as pd

# Load the CSV file
mnist_df = pd.read_csv('/content/sample_data/mnist_train_small.csv')

# Display the first 5 rows
display(mnist_df.head())

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MNISTDataset(Dataset):
    def __init__(self, dataframe):
        self.labels = dataframe.iloc[:, 0].values
        self.images = dataframe.iloc[:, 1:].values.astype('uint8').reshape(-1, 28, 28)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = torch.from_numpy(self.images[idx]).float().unsqueeze(0) / 255.0
        label = torch.tensor(self.labels[idx])
        return image, label

# Create the dataset
mnist_dataset = MNISTDataset(mnist_df)

# Create the DataLoader
mnist_dataloader = DataLoader(mnist_dataset, batch_size=64, shuffle=True)

print("PyTorch Dataset and DataLoader created successfully.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# Function to show a batch of images
def show_images(dataloader):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)
    img_grid = torchvision.utils.make_grid(images)
    img_grid = img_grid.numpy().transpose((1, 2, 0))
    plt.imshow(img_grid)
    plt.axis('off')
    plt.show()

# Visualize a batch of training data
show_images(mnist_dataloader)

In [ ]:
# =========================================================
# 0) Environment & Imports\

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
torch.manual_seed(42)

%config InlineBackend.figure_format = 'retina'

OUT = Path("./outputs")
OUT.mkdir(parents=True, exist_ok=True)

print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)


In [ ]:
# =========================================================
# 1) Define LeNet-5 (for MNIST, 32x32 input)
#    Classic: conv(6,5x5)->pool->conv(16,5x5)->pool->FC(120)->FC(84)->FC(10)
# =========================================================
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)   # 1x32x32 -> 6x28x28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                 # 6x28x28 -> 6x14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)  # 6x14x14 -> 16x10x10
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)                 # 16x10x10 -> 16x5x5

        self.fc1 = nn.Linear(16*5*5, 120)  # 400 -> 120
        self.fc2 = nn.Linear(120, 84)      # 120 -> 84
        self.fc3 = nn.Linear(84, num_classes)  # 84 -> 10

    def forward(self, x):
        x = F.relu(self.conv1(x))  # conv1 out
        x = self.pool1(x)
        x = F.relu(self.conv2(x))  # conv2 out
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = self.fc3(x)
        return logits

model = LeNet5().eval()
print(model)


In [ ]:

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# DataLoader
train_set = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)

for epoch in range(3):
    model.train()
    total_loss, correct = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
    acc = correct / len(train_set)
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_set):.4f}, acc={acc:.4f}")


In [ ]:
# =========================================================
# 2) Data: MNIST (resize to 32x32)
# =========================================================
test_set = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

sample_img, sample_label = next(iter(test_loader))
print("Sample shape:", sample_img.shape, "Label:", sample_label.item())

plt.figure(figsize=(3,3))
plt.title(f"MNIST sample (label={sample_label.item()})")
plt.axis('off')
plt.imshow(sample_img[0,0].numpy(), cmap='gray')
plt.show()


In [ ]:
# =========================================================
# 3) Feature Map Visualization via Hooks (micro-level)
#    - Capture outputs of conv1 and conv2
# =========================================================
model.to('cpu')
feature_maps = {}

def get_hook(name):
    def hook(module, input, output):
        # output: [B, C, H, W]
        feature_maps[name] = output.detach().cpu()
    return hook

h1 = model.conv1.register_forward_hook(get_hook("conv1"))
h2 = model.conv2.register_forward_hook(get_hook("conv2"))

with torch.no_grad():
    _ = model(sample_img)

h1.remove()
h2.remove()

def visualize_feature_maps(fm, n_cols=8, title="Feature Maps"):
    """
    fm: Tensor [B, C, H, W], we visualize the first sample
    """
    fm = fm[0]  # [C, H, W]
    C = fm.shape[0]
    n_rows = int(np.ceil(C / n_cols))
    fm_min = fm.min(dim=1, keepdim=True)[0].min(dim=2, keepdim=True)[0]
    fm_max = fm.max(dim=1, keepdim=True)[0].max(dim=2, keepdim=True)[0]
    fm_norm = (fm - fm_min) / (fm_max - fm_min + 1e-6)

    grid = make_grid(fm_norm.unsqueeze(1), nrow=n_cols, padding=1, normalize=False)
    plt.figure(figsize=(1.6*n_cols, 1.6*n_rows))
    plt.suptitle(title)
    plt.axis('off')
    plt.imshow(grid.permute(1,2,0).numpy().squeeze(), cmap='gray')
    plt.show()

visualize_feature_maps(feature_maps["conv1"], n_cols=6, title="conv1 feature maps (6 channels)")
visualize_feature_maps(feature_maps["conv2"], n_cols=8, title="conv2 feature maps (16 channels)")


In [ ]:
# Save the model in pt
dummy_input = torch.randn(1, 1, 32, 32)
# trace model
traced_model = torch.jit.trace(model, dummy_input)
# save model
traced_model.save('lenet5.pt')


In [ ]:
import os
import math
from pathlib import Path
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from PIL import Image
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# =========================================================
# 1) Load pretrained AlexNet
# =========================================================

model = alexnet(pretrained=True).to(DEVICE).eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
])

print(model)


In [ ]:
# =========================================================
# 2) Load an image (upload or use sample)
# =========================================================
from google.colab import files

uploaded = files.upload()
if len(uploaded) > 0:
    fname = list(uploaded.keys())[0]
    img = Image.open(fname).convert('RGB')
else:
    w, h = 640, 400
    arr = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(h):
        arr[i,:,0] = np.clip(255 * (i/h), 0, 255)
        arr[i,:,1] = np.clip(255 * (1 - i/h), 0, 255)
        arr[i,:,2] = 128
    img = Image.fromarray(arr, 'RGB')

plt.figure(figsize=(4,4)); plt.imshow(img); plt.axis('off'); plt.title("Input image")
plt.show()

inp = preprocess(img).unsqueeze(0).to(DEVICE)  # [1,3,224,224]


In [ ]:
# =========================================================
# 3) Choose layers to hook
#    AlexNet.features
#    0:Conv1, 1:ReLU, 2:MaxPool,
#    3:Conv2, 4:ReLU, 5:MaxPool,
#    6:Conv3, 7:ReLU,
#    8:Conv4, 9:ReLU,
#    10:Conv5, 11:ReLU, 12:MaxPool
# =========================================================
layer_indices = [0,3,6,8,10]
layer_names = {
    0: "conv1", 1: "relu1", 2: "pool1",
    3: "conv2", 4: "relu2", 5: "pool2",
    6: "conv3", 7: "relu3",
    8: "conv4", 9: "relu4",
    10:"conv5", 11:"relu5", 12:"pool5"
}

feature_maps = {}

def make_hook(name):
    def hook(m, x, y):
        # y: [B,C,H,W]
        feature_maps[name] = y.detach().float().cpu()
    return hook

handles = []
for idx in layer_indices:
    m = model.features[idx]
    handles.append(m.register_forward_hook(make_hook(layer_names[idx])))

# Forward once
with torch.no_grad():
    _ = model(inp)

# clear hook
for h in handles:
    h.remove()

list(feature_maps.keys())


In [ ]:
# =========================================================
# 4) Utilities: visualize & save feature map grids
# =========================================================
from torchvision.utils import make_grid
OUTDIR = Path('./alexnet_fmaps'); OUTDIR.mkdir(exist_ok=True, parents=True)
def minmax_norm(t, eps=1e-6):
    # t: [H,W]
    t_min = t.amin(dim=(-2,-1), keepdim=True)
    t_max = t.amax(dim=(-2,-1), keepdim=True)
    return (t - t_min) / (t_max - t_min + eps)

def save_fmap_grid(t, title, ncols=8, limit=None):
    """
    t: Tensor [B, C, H, W]
    """
    t = t[0]  # [C,H,W]
    C, H, W = t.shape
    if limit is not None:
        C = min(C, limit)
        t = t[:C]

    normed = []
    for c in range(C):
        x = t[c]
        x = (x - x.min()) / (x.max() - x.min() + 1e-6)
        normed.append(x.unsqueeze(0))  # [1,H,W]
    grid = make_grid(torch.stack(normed, dim=0), nrow=ncols, padding=2)  # [3?,H',W'] 但我们给的是单通道，所以会是 [C',H',W']

    plt.figure(figsize=(1.6*ncols, 1.6*math.ceil(C/ncols)))
    plt.suptitle(title)
    plt.axis('off')
    plt.imshow(grid.permute(1,2,0).squeeze(), cmap='gray')
    plt.show()

    # save
    outpath = OUTDIR / f"{title.replace(' ','_')}.png"
    plt.imsave(outpath.as_posix(), grid.permute(1,2,0).squeeze(), cmap='gray')
    print("Saved:", outpath)

for k in feature_maps:
    ch = feature_maps[k].shape[1]
    limit = 32 if ch > 32 else None
    save_fmap_grid(feature_maps[k], title=f"{k} ({ch}ch)", ncols=8, limit=limit)


In [ ]:
# =========================================================
# 5) Make a prediction and show the class name
# =========================================================
# Load ImageNet class labels
import json
from google.colab import data_table
import requests

# Updated URL for ImageNet class labels
class_labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
response = requests.get(class_labels_url)
class_labels = [line.strip() for line in response.text.split('\n') if line.strip()]

# Make prediction
model.to(DEVICE) # Move model back to the device if it was moved to cpu
with torch.no_grad():
    outputs = model(inp)
    _, predicted = outputs.max(1)
    predicted_class_index = predicted.item()

# Get the predicted class name
predicted_class_name = class_labels[predicted_class_index]

print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class name: {predicted_class_name}")

In [ ]:
# =========================================================
# 5) Optional: also visualize first-layer kernels directly
# =========================================================
w = model.features[0].weight.detach().cpu()  # [64,3,11,11]

kernels = w.mean(1, keepdim=True)  # [64,1,11,11]
kernels = (kernels - kernels.min()) / (kernels.max() - kernels.min() + 1e-6)
grid = make_grid(kernels, nrow=8, padding=1)

plt.figure(figsize=(12,10))
plt.title("conv1 kernels (channel-mean)")
plt.axis('off')
plt.imshow(grid.permute(1,2,0).squeeze(), cmap='gray')
plt.show()

plt.imsave((OUTDIR/"conv1_kernels.png").as_posix(), grid.permute(1,2,0).squeeze(), cmap='gray')
print("Saved:", OUTDIR/"conv1_kernels.png")


In [ ]:
# Save the model pt
model.to('cpu')
dummy_input = torch.randn(1, 3, 224, 224)
# trace model
traced_model = torch.jit.trace(model, dummy_input)
# save model
traced_model.save('alexnet.pt')


In [ ]:
from torchvision.models import inception_v3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# =========================================================
# 1) Load pretrained Inception
# =========================================================

model = inception_v3(pretrained=True).to(DEVICE).eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
])

print(model)

In [ ]:
# =========================================================
# 2) Load an image (upload or use sample) - Using the same image loading code as before
# =========================================================
from google.colab import files

uploaded = files.upload()
if len(uploaded) > 0:
    fname = list(uploaded.keys())[0]
    img = Image.open(fname).convert('RGB')
else:
    w, h = 640, 400
    arr = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(h):
        arr[i,:,0] = np.clip(255 * (i/h), 0, 255)
        arr[i,:,1] = np.clip(255 * (1 - i/h), 0, 255)
        arr[i,:,2] = 128
    img = Image.fromarray(arr, 'RGB')

plt.figure(figsize=(4,4)); plt.imshow(img); plt.axis('off'); plt.title("Input image")
plt.show()

inp = preprocess(img).unsqueeze(0).to(DEVICE)  # [1,3,224,224]

# =========================================================
# 3) Choose layers to hook
#    Inception V3 layers: Conv2d_1a_3x3, Conv2d_2a_3x3, Conv2d_2b_3x3, Mixed_5b, Mixed_5c, Mixed_5d, Mixed_6a, Mixed_6b, Mixed_6c, Mixed_6d, Mixed_6e, Mixed_7a, Mixed_7b, Mixed_7c
# =========================================================
layer_names = ['Conv2d_1a_3x3', 'Conv2d_2a_3x3', 'Conv2d_2b_3x3', 'Mixed_5b', 'Mixed_5c', 'Mixed_5d', 'Mixed_6a', 'Mixed_6b', 'Mixed_6c', 'Mixed_6d', 'Mixed_6e', 'Mixed_7a', 'Mixed_7b', 'Mixed_7c']

feature_maps = {}

def make_hook(name):
    def hook(m, x, y):
        # y: [B,C,H,W]
        feature_maps[name] = y.detach().float().cpu()
    return hook

handles = []
for name in layer_names:
    m = getattr(model, name)
    handles.append(m.register_forward_hook(make_hook(name)))

# Forward once
with torch.no_grad():
    _ = model(inp)

# clear hook
for h in handles:
    h.remove()

list(feature_maps.keys())

In [ ]:
# =========================================================
# 4) Utilities: visualize & save feature map grids
# =========================================================
from torchvision.utils import make_grid
OUTDIR = Path('./inception_fmaps'); OUTDIR.mkdir(exist_ok=True, parents=True)
def minmax_norm(t, eps=1e-6):
    # t: [H,W]
    t_min = t.amin(dim=(-2,-1), keepdim=True)
    t_max = t.amax(dim=(-2,-1), keepdim=True)
    return (t - t_min) / (t_max - t_min + eps)

def save_fmap_grid(t, title, ncols=8, limit=None):
    """
    t: Tensor [B, C, H, W]
    """
    t = t[0]  # [C,H,W]
    C, H, W = t.shape
    if limit is not None:
        C = min(C, limit)
        t = t[:C]

    normed = []
    for c in range(C):
        x = t[c]
        x = (x - x.min()) / (x.max() - x.min() + 1e-6)
        normed.append(x.unsqueeze(0))  # [1,H,W]
    grid = make_grid(torch.stack(normed, dim=0), nrow=ncols, padding=2)  # [3?,H',W'] 但我们给的是单通道，所以会是 [C',H',W']

    plt.figure(figsize=(1.6*ncols, 1.6*math.ceil(C/ncols)))
    plt.suptitle(title)
    plt.axis('off')
    plt.imshow(grid.permute(1,2,0).squeeze(), cmap='gray')
    plt.show()

    # save
    outpath = OUTDIR / f"{title.replace(' ','_')}.png"
    plt.imsave(outpath.as_posix(), grid.permute(1,2,0).squeeze(), cmap='gray')
    print("Saved:", outpath)

# Visualize feature maps for 'Mixed_5b'
layer_name = 'Mixed_5b'
if layer_name in feature_maps:
    ch = feature_maps[layer_name].shape[1]
    limit = 32 if ch > 32 else None
    save_fmap_grid(feature_maps[layer_name], title=f"{layer_name} ({ch}ch)", ncols=8, limit=limit)
else:
    print(f"Feature maps for '{layer_name}' not found.")

In [ ]:
# =========================================================
# 5) Make a prediction and show the class name
# =========================================================
# Load ImageNet class labels
import json
from google.colab import data_table
import requests

# Updated URL for ImageNet class labels
class_labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
response = requests.get(class_labels_url)
class_labels = [line.strip() for line in response.text.split('\n') if line.strip()]

# Make prediction
model.to(DEVICE) # Move model back to the device if it was moved to cpu
with torch.no_grad():
    outputs = model(inp)
    _, predicted = outputs.max(1)
    predicted_class_index = predicted.item()

# Get the predicted class name
predicted_class_name = class_labels[predicted_class_index]

print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class name: {predicted_class_name}")

In [ ]:
# Save the model pt
model.to('cpu')
dummy_input = torch.randn(1, 3, 224, 224)
# trace model
traced_model = torch.jit.trace(model, dummy_input)
# save model
traced_model.save('inception_v3.pt')

In [ ]:
from torchvision.models import resnet18
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# =========================================================
# 1) Load pretrained Resnet-18
# =========================================================

model = resnet18(pretrained=True).to(DEVICE).eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
])

print(model)


In [ ]:
# =========================================================
# 2) Load an image (upload or use sample) - Using the same image loading code as before
# =========================================================
from google.colab import files

uploaded = files.upload()
if len(uploaded) > 0:
    fname = list(uploaded.keys())[0]
    img = Image.open(fname).convert('RGB')
else:
    w, h = 640, 400
    arr = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(h):
        arr[i,:,0] = np.clip(255 * (i/h), 0, 255)
        arr[i,:,1] = np.clip(255 * (1 - i/h), 0, 255)
        arr[i,:,2] = 128
    img = Image.fromarray(arr, 'RGB')

plt.figure(figsize=(4,4)); plt.imshow(img); plt.axis('off'); plt.title("Input image")
plt.show()

inp = preprocess(img).unsqueeze(0).to(DEVICE)  # [1,3,224,224]

# =========================================================
# 3) Choose layers to hook
#    ResNet18 layers: conv1, layer1, layer2, layer3, layer4
# =========================================================
layer_names = ['conv1', 'layer1', 'layer2', 'layer3', 'layer4']

feature_maps = {}

def make_hook(name):
    def hook(m, x, y):
        # y: [B,C,H,W]
        feature_maps[name] = y.detach().float().cpu()
    return hook

handles = []
for name in layer_names:
    m = getattr(model, name)
    handles.append(m.register_forward_hook(make_hook(name)))

# Forward once
with torch.no_grad():
    _ = model(inp)

# clear hook
for h in handles:
    h.remove()

list(feature_maps.keys())

In [ ]:
# =========================================================
# 4) Utilities: visualize & save feature map grids
# =========================================================
from torchvision.utils import make_grid
OUTDIR = Path('./resnet_fmaps'); OUTDIR.mkdir(exist_ok=True, parents=True)
def minmax_norm(t, eps=1e-6):
    # t: [H,W]
    t_min = t.amin(dim=(-2,-1), keepdim=True)
    t_max = t.amax(dim=(-2,-1), keepdim=True)
    return (t - t_min) / (t_max - t_min + eps)

def save_fmap_grid(t, title, ncols=8, limit=None):
    """
    t: Tensor [B, C, H, W]
    """
    t = t[0]  # [C,H,W]
    C, H, W = t.shape
    if limit is not None:
        C = min(C, limit)
        t = t[:C]

    normed = []
    for c in range(C):
        x = t[c]
        x = (x - x.min()) / (x.max() - x.min() + 1e-6)
        normed.append(x.unsqueeze(0))  # [1,H,W]
    grid = make_grid(torch.stack(normed, dim=0), nrow=ncols, padding=2)
    plt.figure(figsize=(1.6*ncols, 1.6*math.ceil(C/ncols)))
    plt.suptitle(title)
    plt.axis('off')
    plt.imshow(grid.permute(1,2,0).squeeze(), cmap='gray')
    plt.show()

    # save
    outpath = OUTDIR / f"{title.replace(' ','_')}.png"
    plt.imsave(outpath.as_posix(), grid.permute(1,2,0).squeeze(), cmap='gray')
    print("Saved:", outpath)

# Visualize feature maps for 'layer1'
layer_name = 'conv1'
if layer_name in feature_maps:
    ch = feature_maps[layer_name].shape[1]
    limit = 32 if ch > 32 else None
    save_fmap_grid(feature_maps[layer_name], title=f"{layer_name} ({ch}ch)", ncols=8, limit=limit)
else:
    print(f"Feature maps for '{layer_name}' not found.")

In [ ]:
# =========================================================
# 5) Make a prediction and show the class name
# =========================================================
# Load ImageNet class labels
import json
from google.colab import data_table
import requests

# Updated URL for ImageNet class labels
class_labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
response = requests.get(class_labels_url)
class_labels = [line.strip() for line in response.text.split('\n') if line.strip()]

# Make prediction
model.to(DEVICE) # Move model back to the device if it was moved to cpu
with torch.no_grad():
    outputs = model(inp)
    _, predicted = outputs.max(1)
    predicted_class_index = predicted.item()

# Get the predicted class name
predicted_class_name = class_labels[predicted_class_index]

print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class name: {predicted_class_name}")

In [ ]:
# Save the model pt
model.to('cpu')
dummy_input = torch.randn(1, 3, 224, 224)
# trace model
traced_model = torch.jit.trace(model, dummy_input)
# save model
traced_model.save('resnet18.pt')